In [ ]:
from pathlib import Path
import h5py
import numpy as np
import cv2

# ============================================================
# Project structure (relative to script location)
# ============================================================
PROJECT_DIR = Path.cwd()
RESULTS_DIR = PROJECT_DIR / "results"
RESULTS_DIR.mkdir(exist_ok=True)

# Name of the recording (same as AEDAT stem)
BASE_NAME = "dvSave_test"


Visulizer 1: Project window in a Dome and don't morph the output.

In [6]:
from pathlib import Path
import h5py
import numpy as np
import cv2

# ============================================================
# Project structure (relative to script location)
# ============================================================
PROJECT_DIR = Path.cwd()
RESULTS_DIR = PROJECT_DIR / "results"
RESULTS_DIR.mkdir(exist_ok=True)

# Name of the recording (same as AEDAT stem)
BASE_NAME = "dvSave_test"

# Input panorama from results folder
IN_H5 = RESULTS_DIR / f"{BASE_NAME}_results.h5"

# Output dome video (also in results)
OUT_MP4 = RESULTS_DIR / f"{BASE_NAME}_dome.mp4"

# Convert to string for OpenCV/h5py
IN_H5 = str(IN_H5)
OUT_MP4 = str(OUT_MP4)

print("Reading panorama:", IN_H5)
print("Saving dome video:", OUT_MP4)

# Input canvas size (matches your panorama)
H_IN, W_IN = 1440, 1920

# Output image size (hemi-dome view)
H_OUT, W_OUT = 900, 900   # square works best for a dome

WINDOW = 30_000
FPS = 30

CLIP_ACC = 100
CLIP_WIN = 5

# Dome (half-sphere) parameters
CENTER_X = (W_IN - 1) * 0.5
CENTER_Y = (H_IN - 1) * 0.5

# Max radius in input pixels that corresponds to the dome horizon (90° from center).
# If None, use max distance to corners (often too big; set manually for nicer dome).
R_MAX = None

# If True, angles increase clockwise (image coords friendly)
CLOCKWISE = True

# Optional: flip angular direction if your dome spins the wrong way
FLIP_THETA = False
# ============================


def counts_image_from_xy(xy, H, W):
    """Return (H,W) int32 count image from xy array (x,y)."""
    if xy.size == 0:
        return np.zeros((H, W), dtype=np.int32)

    x = xy[:, 0].astype(np.int64)
    y = xy[:, 1].astype(np.int64)

    m = (x >= 0) & (x < W) & (y >= 0) & (y < H)
    x = x[m]; y = y[m]
    if x.size == 0:
        return np.zeros((H, W), dtype=np.int32)

    idx = y * W + x
    return np.bincount(idx, minlength=H * W).reshape(H, W).astype(np.int32)


def compute_rmax_default(w, h, cx, cy):
    corners = np.array([[0, 0],
                        [w - 1, 0],
                        [0, h - 1],
                        [w - 1, h - 1]], dtype=np.float64)
    dx = corners[:, 0] - cx
    dy = corners[:, 1] - cy
    return float(np.max(np.sqrt(dx * dx + dy * dy)))


def hemispherical_project_xy(xy, cx, cy, out_h, out_w, r_max,
                             clockwise=True, flip_theta=False):
    """
    Project (x,y) around (cx,cy) into a HALF-SPHERE (hemidome) and output
    as a top-down azimuthal projection (a circular dome image).

    Mapping:
      r_img = sqrt(dx^2 + dy^2)
      theta = atan2(dy, dx)  (azimuth)
      phi   = (r_img / r_max) * (pi/2)   (polar angle from zenith), clamped [0, pi/2]

    Then use an azimuthal equidistant projection:
      rho = phi / (pi/2) * R_out   (0..R_out)
      u = cx_out + rho*cos(theta)
      v = cy_out - rho*sin(theta)   (minus to keep +v downward)

    Notes:
    - This treats your input radius as proportional to angle off-center.
      It’s a reasonable "half-sphere" visualization when you don't have intrinsics.
    - For physically-correct dome from a camera, you'd use rays + spherical coords.
    """
    if xy.size == 0:
        return xy.astype(np.int64)

    x = xy[:, 0].astype(np.float64)
    y = xy[:, 1].astype(np.float64)

    dx = x - cx
    dy = y - cy

    # image coords: +y down. "clockwise" usually feels natural.
    if clockwise:
        dy = -dy

    theta = np.arctan2(dy, dx)  # [-pi, pi]
    if flip_theta:
        theta = -theta
    theta = np.mod(theta, 2.0 * np.pi)

    r = np.sqrt(dx * dx + dy * dy)

    # keep only within r_max (inside dome)
    m = (r >= 0) & (r <= r_max)
    if not np.any(m):
        return np.zeros((0, 2), dtype=np.int64)

    r = r[m]
    theta = theta[m]

    # map input radius to polar angle phi in [0, pi/2]
    phi = (r / (r_max + 1e-12)) * (0.5 * np.pi)
    phi = np.clip(phi, 0.0, 0.5 * np.pi)

    # azimuthal equidistant: rho proportional to phi
    R_out = 0.5 * min(out_w - 1, out_h - 1)
    rho = (phi / (0.5 * np.pi)) * R_out

    cx_out = (out_w - 1) * 0.5
    cy_out = (out_h - 1) * 0.5

    u = cx_out + rho * np.cos(theta)
    v = cy_out - rho * np.sin(theta)  # - keeps image v positive downward

    # bounds
    u = np.floor(u).astype(np.int64)
    v = np.floor(v).astype(np.int64)

    mm = (u >= 0) & (u < out_w) & (v >= 0) & (v < out_h)
    u = u[mm]
    v = v[mm]

    return np.column_stack([u, v])


def main():
    r_max = R_MAX
    if r_max is None:
        r_max = compute_rmax_default(W_IN, H_IN, CENTER_X, CENTER_Y)
    print(f"Hemisphere projection center=({CENTER_X:.1f},{CENTER_Y:.1f}) r_max={r_max:.2f} "
          f"out=(H={H_OUT},W={W_OUT})")

    with h5py.File(IN_H5, "r") as h5:
        dpos = h5["pos"]  # (N,3) x,y,t
        dneg = h5["neg"]  # (M,3) x,y,t

        tpos = dpos[:, 2] if dpos.shape[0] else np.array([], dtype=np.int64)
        tneg = dneg[:, 2] if dneg.shape[0] else np.array([], dtype=np.int64)

        t_min = min(tpos[0] if len(tpos) else 10**30,
                    tneg[0] if len(tneg) else 10**30)
        t_max = max(tpos[-1] if len(tpos) else 0,
                    tneg[-1] if len(tneg) else 0)

        times = np.arange(int(t_min), int(t_max), int(WINDOW))
        print("frames:", len(times), "WINDOW:", WINDOW, "duration(us):", int(t_max - t_min))

        pos_i0 = np.searchsorted(tpos, times, side="left") if len(tpos) else np.zeros(len(times), dtype=int)
        pos_i1 = np.searchsorted(tpos, times + WINDOW, side="left") if len(tpos) else np.zeros(len(times), dtype=int)
        neg_j0 = np.searchsorted(tneg, times, side="left") if len(tneg) else np.zeros(len(times), dtype=int)
        neg_j1 = np.searchsorted(tneg, times + WINDOW, side="left") if len(tneg) else np.zeros(len(times), dtype=int)

        # Persistent accumulation (in HEMI output space)
        acc_pos = np.zeros((H_OUT, W_OUT), dtype=np.int32)
        acc_neg = np.zeros((H_OUT, W_OUT), dtype=np.int32)

        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        vw = cv2.VideoWriter(OUT_MP4, fourcc, FPS, (W_OUT, H_OUT), isColor=True)

        for k in range(len(times)):
            i0, i1 = int(pos_i0[k]), int(pos_i1[k])
            j0, j1 = int(neg_j0[k]), int(neg_j1[k])

            pos_xy = np.asarray(dpos[i0:i1, :2]) if i1 > i0 else np.zeros((0, 2), dtype=np.int64)
            neg_xy = np.asarray(dneg[j0:j1, :2]) if j1 > j0 else np.zeros((0, 2), dtype=np.int64)

            # --- HEMISPHERE (HALF-SPHERE) PROJECTION STEP ---
            pos_uv = hemispherical_project_xy(
                pos_xy, CENTER_X, CENTER_Y, H_OUT, W_OUT, r_max,
                clockwise=CLOCKWISE, flip_theta=FLIP_THETA
            )
            neg_uv = hemispherical_project_xy(
                neg_xy, CENTER_X, CENTER_Y, H_OUT, W_OUT, r_max,
                clockwise=CLOCKWISE, flip_theta=FLIP_THETA
            )

            win_pos = counts_image_from_xy(pos_uv, H_OUT, W_OUT)
            win_neg = counts_image_from_xy(neg_uv, H_OUT, W_OUT)

            acc_pos += win_pos
            acc_neg += win_neg

            out_pos = np.maximum(acc_pos - win_pos, 0)
            out_neg = np.maximum(acc_neg - win_neg, 0)

            out_pos8 = (np.clip(out_pos, 0, CLIP_ACC) * (255.0 / CLIP_ACC)).astype(np.uint8)
            out_neg8 = (np.clip(out_neg, 0, CLIP_ACC) * (255.0 / CLIP_ACC)).astype(np.uint8)

            win_pos8 = (np.clip(win_pos, 0, CLIP_WIN) * (255.0 / CLIP_WIN)).astype(np.uint8)
            win_neg8 = (np.clip(win_neg, 0, CLIP_WIN) * (255.0 / CLIP_WIN)).astype(np.uint8)

            bgr = np.zeros((H_OUT, W_OUT, 3), dtype=np.uint8)
            bgr[..., 2] = out_pos8  # R
            bgr[..., 0] = out_neg8  # B

            bgr[..., 1] = np.maximum(bgr[..., 1], win_pos8)       # G for current pos
            bgr[..., 0] = np.maximum(bgr[..., 0], win_neg8)       # B for current neg
            bgr[..., 1] = np.maximum(bgr[..., 1], win_neg8 // 2)  # add some G -> cyan-ish

            vw.write(bgr)

            if k % 200 == 0:
                print(f"k={k}/{len(times)} acc_pos_max={acc_pos.max()} acc_neg_max={acc_neg.max()}")

        vw.release()

    print("saved:", OUT_MP4)


if __name__ == "__main__":
    main()


Reading panorama: /home/maurice/Desktop/Master_Test/results/dvSave_test_results.h5
Saving dome video: /home/maurice/Desktop/Master_Test/results/dvSave_test_dome.mp4
Hemisphere projection center=(959.5,719.5) r_max=1199.30 out=(H=900,W=900)
frames: 1462 WINDOW: 30000 duration(us): 43850958
k=0/1462 acc_pos_max=146 acc_neg_max=92
k=200/1462 acc_pos_max=1574 acc_neg_max=1588
k=400/1462 acc_pos_max=3125 acc_neg_max=1897
k=600/1462 acc_pos_max=3763 acc_neg_max=3085
k=800/1462 acc_pos_max=5684 acc_neg_max=3104
k=1000/1462 acc_pos_max=6439 acc_neg_max=4233
k=1200/1462 acc_pos_max=9378 acc_neg_max=6915
k=1400/1462 acc_pos_max=14575 acc_neg_max=11135
saved: /home/maurice/Desktop/Master_Test/results/dvSave_test_dome.mp4


Visualizer 2: Projection in Dome with morph

In [7]:
IN_H5   = str(RESULTS_DIR / f"{BASE_NAME}_results.h5")
OUT_MP4 = str(RESULTS_DIR / f"{BASE_NAME}_dome_morph.mp4")
# ============================

# Input canvas size (matches your panorama)
H_IN, W_IN = 1440, 1920

# Output image size (dome image)
H_OUT, W_OUT = 900, 900   # square is best for a dome

WINDOW = 30_000
FPS = 30

CLIP_ACC = 50
CLIP_WIN = 5

YAW_SPAN_DEG   = 180.0
PITCH_SPAN_DEG = 90.0

CX_IN = (W_IN - 1) * 0.5
CY_IN = (H_IN - 1) * 0.5

CLOCKWISE = True
FLIP_YAW  = False
FLIP_PITCH = False


def counts_image_from_xy(xy, H, W):
    if xy.size == 0:
        return np.zeros((H, W), dtype=np.int32)

    x = xy[:, 0].astype(np.int64)
    y = xy[:, 1].astype(np.int64)

    m = (x >= 0) & (x < W) & (y >= 0) & (y < H)
    x = x[m]; y = y[m]
    if x.size == 0:
        return np.zeros((H, W), dtype=np.int32)

    idx = y * W + x
    return np.bincount(idx, minlength=H * W).reshape(H, W).astype(np.int32)


def hemi_project_from_panorama_xy(xy,
                                 w_in, h_in, cx_in, cy_in,
                                 out_w, out_h,
                                 yaw_span_deg, pitch_span_deg,
                                 clockwise=True,
                                 flip_yaw=False,
                                 flip_pitch=False):
    if xy.size == 0:
        return np.zeros((0, 2), dtype=np.int64)

    x = xy[:, 0].astype(np.float64)
    y = xy[:, 1].astype(np.float64)

    nx = (x - cx_in) / (w_in - 1)
    ny = (y - cy_in) / (h_in - 1)

    yaw_span = np.deg2rad(yaw_span_deg)
    pitch_span = np.deg2rad(pitch_span_deg)

    yaw = nx * yaw_span
    pitch = -ny * pitch_span

    if flip_yaw:
        yaw = -yaw
    if flip_pitch:
        pitch = -pitch

    cp = np.cos(pitch)
    sp = np.sin(pitch)
    cy = np.cos(yaw)
    sy = np.sin(yaw)

    dx = cp * sy
    dy = sp
    dz = cp * cy

    alpha = np.arctan2(np.sqrt(dx*dx + dy*dy), dz)

    m = (dz > 0) & (alpha <= 0.5 * np.pi)
    if not np.any(m):
        return np.zeros((0, 2), dtype=np.int64)

    dx = dx[m]; dy = dy[m]
    alpha = alpha[m]

    theta = np.arctan2(dy, dx)
    if clockwise:
        theta = -theta

    R_out = 0.5 * min(out_w - 1, out_h - 1)
    rho = (alpha / (0.5 * np.pi)) * R_out

    cx_out = (out_w - 1) * 0.5
    cy_out = (out_h - 1) * 0.5

    u = cx_out + rho * np.cos(theta)
    v = cy_out + rho * np.sin(theta)

    u = np.floor(u).astype(np.int64)
    v = np.floor(v).astype(np.int64)

    mm = (u >= 0) & (u < out_w) & (v >= 0) & (v < out_h)
    u = u[mm]; v = v[mm]

    return np.column_stack([u, v])


def main():
    print(f"Input panorama: {IN_H5}")
    print(f"Output dome MP4: {OUT_MP4}")
    print(f"Input panorama size: {W_IN}x{H_IN}  -> Dome: {W_OUT}x{H_OUT}")
    print(f"Yaw span: {YAW_SPAN_DEG} deg, Pitch span: {PITCH_SPAN_DEG} deg")

    with h5py.File(IN_H5, "r") as h5:
        dpos = h5["pos"]
        dneg = h5["neg"]

        tpos = dpos[:, 2] if dpos.shape[0] else np.array([], dtype=np.int64)
        tneg = dneg[:, 2] if dneg.shape[0] else np.array([], dtype=np.int64)

        t_min = min(tpos[0] if len(tpos) else 10**30,
                    tneg[0] if len(tneg) else 10**30)
        t_max = max(tpos[-1] if len(tpos) else 0,
                    tneg[-1] if len(tneg) else 0)

        times = np.arange(int(t_min), int(t_max), int(WINDOW))
        print("frames:", len(times), "WINDOW:", WINDOW, "duration(us):", int(t_max - t_min))

        pos_i0 = np.searchsorted(tpos, times, side="left") if len(tpos) else np.zeros(len(times), dtype=int)
        pos_i1 = np.searchsorted(tpos, times + WINDOW, side="left") if len(tpos) else np.zeros(len(times), dtype=int)
        neg_j0 = np.searchsorted(tneg, times, side="left") if len(tneg) else np.zeros(len(times), dtype=int)
        neg_j1 = np.searchsorted(tneg, times + WINDOW, side="left") if len(tneg) else np.zeros(len(times), dtype=int)

        acc_pos = np.zeros((H_OUT, W_OUT), dtype=np.int32)
        acc_neg = np.zeros((H_OUT, W_OUT), dtype=np.int32)

        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        vw = cv2.VideoWriter(OUT_MP4, fourcc, FPS, (W_OUT, H_OUT), isColor=True)

        for k in range(len(times)):
            i0, i1 = int(pos_i0[k]), int(pos_i1[k])
            j0, j1 = int(neg_j0[k]), int(neg_j1[k])

            pos_xy = np.asarray(dpos[i0:i1, :2]) if i1 > i0 else np.zeros((0, 2), dtype=np.int64)
            neg_xy = np.asarray(dneg[j0:j1, :2]) if j1 > j0 else np.zeros((0, 2), dtype=np.int64)

            pos_uv = hemi_project_from_panorama_xy(
                pos_xy, W_IN, H_IN, CX_IN, CY_IN, W_OUT, H_OUT,
                YAW_SPAN_DEG, PITCH_SPAN_DEG,
                clockwise=CLOCKWISE, flip_yaw=FLIP_YAW, flip_pitch=FLIP_PITCH
            )
            neg_uv = hemi_project_from_panorama_xy(
                neg_xy, W_IN, H_IN, CX_IN, CY_IN, W_OUT, H_OUT,
                YAW_SPAN_DEG, PITCH_SPAN_DEG,
                clockwise=CLOCKWISE, flip_yaw=FLIP_YAW, flip_pitch=FLIP_PITCH
            )

            win_pos = counts_image_from_xy(pos_uv, H_OUT, W_OUT)
            win_neg = counts_image_from_xy(neg_uv, H_OUT, W_OUT)

            acc_pos += win_pos
            acc_neg += win_neg

            out_pos = np.maximum(acc_pos - win_pos, 0)
            out_neg = np.maximum(acc_neg - win_neg, 0)

            out_pos8 = (np.clip(out_pos, 0, CLIP_ACC) * (255.0 / CLIP_ACC)).astype(np.uint8)
            out_neg8 = (np.clip(out_neg, 0, CLIP_ACC) * (255.0 / CLIP_ACC)).astype(np.uint8)

            win_pos8 = (np.clip(win_pos, 0, CLIP_WIN) * (255.0 / CLIP_WIN)).astype(np.uint8)
            win_neg8 = (np.clip(win_neg, 0, CLIP_WIN) * (255.0 / CLIP_WIN)).astype(np.uint8)

            bgr = np.zeros((H_OUT, W_OUT, 3), dtype=np.uint8)
            bgr[..., 2] = out_pos8
            bgr[..., 0] = out_neg8
            bgr[..., 1] = np.maximum(bgr[..., 1], win_pos8)
            bgr[..., 0] = np.maximum(bgr[..., 0], win_neg8)
            bgr[..., 1] = np.maximum(bgr[..., 1], win_neg8 // 2)

            vw.write(bgr)

            if k % 200 == 0:
                print(f"k={k}/{len(times)} acc_pos_max={acc_pos.max()} acc_neg_max={acc_neg.max()}")

        vw.release()

    print("saved:", OUT_MP4)


if __name__ == "__main__":
    main()

Input panorama: /home/maurice/Desktop/Master_Test/results/dvSave_test_results.h5
Output dome MP4: /home/maurice/Desktop/Master_Test/results/dvSave_test_dome_morph.mp4
Input panorama size: 1920x1440  -> Dome: 900x900
Yaw span: 180.0 deg, Pitch span: 90.0 deg
frames: 1462 WINDOW: 30000 duration(us): 43850958
k=0/1462 acc_pos_max=168 acc_neg_max=114
k=200/1462 acc_pos_max=1820 acc_neg_max=1619
k=400/1462 acc_pos_max=2838 acc_neg_max=1840
k=600/1462 acc_pos_max=3833 acc_neg_max=3024
k=800/1462 acc_pos_max=4370 acc_neg_max=3068
k=1000/1462 acc_pos_max=5767 acc_neg_max=4158
k=1200/1462 acc_pos_max=8524 acc_neg_max=6329
k=1400/1462 acc_pos_max=12821 acc_neg_max=9052
saved: /home/maurice/Desktop/Master_Test/results/dvSave_test_dome_morph.mp4
